In [1]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

from efficientdet.dataset import CocoDataset, collater, get_train_transform, get_val_transform
from backbone import EfficientDetBackbone
from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess, boolean_string

In [4]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()

if cuda_available:
    print("CUDA is available. GPU can be used.")
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Only CPU will be used.")

CUDA is available. GPU can be used.
Number of CUDA devices: 1
Current CUDA device: 0
CUDA device name: NVIDIA GeForce GTX 1060


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

project_name = 'taco'  # also the folder name of the dataset that under data_path folder
train_set = 'train'
val_set = 'val'
num_gpus = 1

# replace with your own path
data_path = 'datasets/'

In [3]:
import yaml

with open(f'projects/{project_name}.yml', 'r') as f:
    params = yaml.safe_load(f)

obj_list = params['obj_list']

In [ ]:
!mkdir -p pre-trained_weights
!wget -nc https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch/releases/download/1.0/efficientdet-d0.pth -O pre-trained_weights/efficientdet-d0.pth

In [6]:
# Train head only
!python train.py -c 0 -p taco --head_only True --lr 1e-3 --batch_size 16 --load_weights pre-trained_weights/efficientdet-d0.pth --num_epochs 50 --save_interval 10 --debug True

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
[Warning] Ignoring Error(s) in loading state_dict for EfficientDetBackbone:
	size mismatch for classifier.header.pointwise_conv.conv.weight: copying a param with shape torch.Size([810, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([540, 64, 1, 1]).
	size mismatch for classifier.header.pointwise_conv.conv.bias: copying a param with shape torch.Size([810]) from checkpoint, the shape in current model is torch.Size([540]).
[Warning] Don't panic if you see this, this might be because you load a pretrained weights with different number of classes. The rest of the weights should be loaded already.
[Info] loaded weights: efficientdet-d0.pth, resuming checkpoint from step: 0
[Info] freezed backbone


c:\Users\itsd\Yet-Another-EfficientDet-Pytorch\.venv\Lib\site-packages\torch\utils\data\dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
c:\Users\itsd\Yet-Another-EfficientDet-Pytorch\train.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weigh

In [ ]:
# Train whole model
!python train.py -c 0 -p taco --head_only False --lr 1e-4 --batch_size 8 --load_weights logs/taco/efficientdet-d0_49_3250.pth --num_epochs 100 --save_interval 50 --debug True

In [ ]:
import glob

weight_file = glob.glob('logs/taco/*.pth')
weight_file.sort()

!python coco_eval.py -c 0 -p taco -w "logs/taco/{weight_file[-1]}" --use_custom_dataset True

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

compound_coef = 0
force_input_size = None  # set None to use default size

threshold = 0.2
iou_threshold = 0.2

use_cuda = True
use_float16 = False
cudnn.fastest = True
cudnn.benchmark = True

obj_list = ['Trash']  # Replace with your actual object list

# tf bilinear interpolation is different from any other's, just make do
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
input_size = input_sizes[compound_coef] if force_input_size is None else force_input_size

model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),
                             ratios=eval(params['anchors_ratios']), scales=eval(params['anchors_scales']))

model.load_state_dict(torch.load(f'logs/taco/{weight_file[-1]}'))
model.requires_grad_(False)
model.eval()

if use_cuda:
    model = model.cuda()

if use_float16:
    model = model.half()

# Load and preprocess image
img_path = 'datasets/taco/images/batch_10000062_jpg.rf.23a022fa34f7c6d7d5369d59ca891b94.jpg'
ori_imgs, framed_imgs, framed_metas = preprocess(img_path, max_size=input_size)

if use_cuda:
    x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
else:
    x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)

x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

with torch.no_grad():
    features, regression, classification, anchors = model(x)

    regressBoxes = BBoxTransform()
    clipBoxes = ClipBoxes()

    out = postprocess(x,
                      anchors, regression, classification,
                      regressBoxes, clipBoxes,
                      threshold, iou_threshold)

out = invert_affine(framed_metas, out)
img_show = ori_imgs[0].copy()

for j in range(len(out[0]['rois'])):
    x1, y1, x2, y2 = out[0]['rois'][j].astype(int)
    obj = obj_list[out[0]['class_ids'][j]]
    score = float(out[0]['scores'][j])
    plot_one_box(img_show, [x1, y1, x2, y2], label=obj, score=score,
                 color=color_list[get_index_label(obj, obj_list)])

plt.figure(figsize=(15, 15))
plt.imshow(img_show)
plt.axis('off')
plt.show()